<a href="https://colab.research.google.com/github/Precillieo/dsn20/blob/master/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSN** **2020**

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from mlxtend.classifier import StackingClassifier
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions 

from lightgbm import LGBMClassifier
!pip install rgf_python
from sklearn import multioutput
from rgf.sklearn import RGFClassifier
!pip install catboost
!pip install xgboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import warnings
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


     |████████████████████████████████| 768kB 2.8MB/s 
     |████████████████████████████████| 66.1MB 56kB/s 


In [2]:
train= pd.read_csv('/content/drive/My Drive/Bootcamp2020/Train.csv')
test= pd.read_csv("/content/drive/My Drive/Bootcamp2020/Test.csv")

In [3]:
train.head(10)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.240,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.2820,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.0000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.000,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.0000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.190,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
5,Apcnt_1000020,3724.0,NaN,0.0000,0.0,0.0000,192944.0,9244585.0,6761209.0,67963357.0,32902009.0,NaN,976390.0,2114215.0,16576457,NaN,0.0,0.0,0.0,0.0,0.0,8.385,NaN,NaN,4.845,2590.5,22083.0,14722.0,3772.0,22083.0,NaN,NaN,3858.0,31.0000,0.0,NaN,10.0,60.0,0.0,0.0,NaN,NaN,0.030558,12.12,0.382960,NaN,0.0,lending,NaN,0.000000,NaN,no
6,Apcnt_1000024,3350.0,0.24205,0.3132,0.0,2.8188,NaN,580388.0,NaN,96472.0,580388.0,631347.0,NaN,0.0,21973443,91602498.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,822.0,NaN,NaN,426.0,13413.0,NaN,13413.0,36.0,14.3334,0.0,NaN,12.0,2.0,2.0,0.0,153.8745,NaN,0.660000,1.01,0.513504,NaN,0.0,charge,NaN,2.146512,0.004390,yes
7,Apcnt_1000028,3416.0,0.87020,0.0570,0.0,0.0000,17505.0,186331.0,186331.0,1484813.0,326760.0,50959.0,1284089.0,5633109.0,11564970,18211813.0,0.0,0.0,0.0,1.0,1.0,124.980,8267.655,7755.00,82.680,1498.5,1734.0,1156.0,548.0,15240.0,182.0,NaN,460.0,5.5000,0.0,1.0,10.0,6.0,12.0,0.0,NaN,149.0190,0.137500,3.03,0.635960,NaN,0.0,charge,155.295994,0.946074,0.128389,no
8,Apcnt_1000036,3376.0,0.29180,0.5586,0.0,0.0000,18283.0,1105149.0,18283.0,115533.0,1557167.0,752715.0,97639.0,427511.0,11564970,NaN,0.0,0.0,0.0,1.0,2.0,126.495,NaN,NaN,126.495,1597.5,3195.0,2130.0,182.0,3741.0,1034.0,NaN,70.0,34.8334,1.0,NaN,14.0,0.0,8.0,0.0,NaN,102.2625,0.471427,4.04,0.410880,0.0,0.0,charge,NaN,0.641601,0.009990,no
9,Apcnt_1000040,3310.0,0.24395,0.1562,0.0,0.0000,NaN,0.0,NaN,NaN,0.0,208115.0,NaN,NaN,11564970,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,502.5,1278.0,852.0,182.0,9126.0,NaN,273.0,22.0,4.1666,0.0,1.0,4.0,0.0,4.0,0.0,150.0000,NaN,0.825000,1.01,0.800000,NaN,NaN,charge

In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Applicant_ID  24000 non-null  object 
 1   form_field1   22890 non-null  float64
 2   form_field2   22291 non-null  float64
 3   form_field3   23854 non-null  float64
 4   form_field4   23854 non-null  float64
 5   form_field5   23854 non-null  float64
 6   form_field6   18396 non-null  float64
 7   form_field7   21769 non-null  float64
 8   form_field8   18396 non-null  float64
 9   form_field9   20600 non-null  float64
 10  form_field10  23853 non-null  float64
 11  form_field11  10602 non-null  float64
 12  form_field12  19817 non-null  float64
 13  form_field13  21537 non-null  float64
 14  form_field14  24000 non-null  int64  
 15  form_field15  14408 non-null  float64
 16  form_field16  18526 non-null  float64
 17  form_field17  19305 non-null  float64
 18  form_field18  19631 non-nu

In [7]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [8]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [9]:
train.columns

Index(['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 'form_field47',
       'form_field48', 'form_field49', 'form_field50', 'default_status'],
      dtype='object')

In [10]:
test.columns

Index(['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 'form_field47',
       'form_field48', 'form_field49', 'form_field50'],
      dtype='object')

In [11]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,2.457900e+04,4.610500e+04,5.011100e+04,5.600000e+04,3.352500e+04,42964.000000,44849.000000,45598.000000,55996.000000,55645.000000,40146.000000,35600.000000,27877.000000,42703.000000,50550.000000,48562.000000,46701.000000,55645.000000,55645.000000,30491.000000,16592.000000,50550.000000,54744.000000,55645.000000,32852.000000,54005.000000,50550.000000,55645.000000,51789.000000,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,2.054715e+06,2.771113e+06,6.009944e+06,6.914875e+07,1.145041e+08,0.230425,0.265692,0.271832,0.948800,1.031593,86.500380,6628.316381,6452.875072,104.558516,2624.555935,10106.554899,6190.516948,17274.315141,16518.540695,5213.246663,10857.565755,938.345321,13.926583,0.230245,0.199866,12.859069,8.480435,8.601420,0.090849,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,4.912980e+06,4.544838e+06,2.335893e+07,2.713995e+09,1.153842e+08,0.621833,0.720198,0.725434,1.736454,1.785122,53.445528,5661.059297,5739.968774,436.670719,1964.226548,8796.472481,5390.915344,26930.742020,9666.183962,3124.491488,6049.678489,1044.597044,15.208396,0.683063,0.574002,10.621984,9.328810,7.969215,0.423802,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395430e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166600,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,2.172565e+05,2.758010e+05,1.921660e+05,1.657646e+07,5.007636e+07,0.000000,0.000000,0.000000,0.000000,0.000000,40.305000,1068.667500,797.145000,14.865000,1231.500000,3375.000000,2068.000000,60.000000,9765.000000,2312.000000,6297.000000,194.000000,2.333400,0.000000,0.000000,6.000000,2.000000,4.000000,0.000000,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,6.986440e+05,1.163110e+06,1.605792e+06,2.814143e+07,8.442428e+07,0.000000,0.000000,0.000000,0.000000,0.000000,89.167500,5976.225000,5548.935000,60.660000,2203.500000,8214.000000,5110.000000,486.000000,15057.000000,5658.000000,10584.000000,598.000000,7.833400,0.000000,0.000000,10.000000,6.000000,6.000000,0.000000,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552

## **DEALING** **WITH** **MISSING** **VALUES**

In [12]:
columns= ('form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field17', 'form_field15', 'form_field16', 'form_field18', 'form_field19', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field48','form_field49','form_field50')
for cod in columns:
  test[cod]= test[cod].fillna(-9999)

test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [13]:
cols= ('form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field17', 'form_field15', 'form_field16', 'form_field18', 'form_field19', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field48','form_field49','form_field50')
for col in cols:
  train[col]= train[col].fillna(-9999)

train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [14]:
train.dtypes

Applicant_ID       object
form_field1       float64
form_field2       float64
form_field3       float64
form_field4       float64
form_field5       float64
form_field6       float64
form_field7       float64
form_field8       float64
form_field9       float64
form_field10      float64
form_field11      float64
form_field12      float64
form_field13      float64
form_field14        int64
form_field15      float64
form_field16      float64
form_field17      float64
form_field18      float64
form_field19      float64
form_field20      float64
form_field21      float64
form_field22      float64
form_field23      float64
form_field24      float64
form_field25      float64
form_field26      float64
form_field27      float64
form_field28      float64
form_field29      float64
form_field30      float64
form_field31      float64
form_field32      float64
form_field33      float64
form_field34      float64
form_field35      float64
form_field36      float64
form_field37      float64
form_field38

In [15]:
test.dtypes

Applicant_ID     object
form_field1     float64
form_field2     float64
form_field3     float64
form_field4     float64
form_field5     float64
form_field6     float64
form_field7     float64
form_field8     float64
form_field9     float64
form_field10    float64
form_field11    float64
form_field12    float64
form_field13    float64
form_field14      int64
form_field15    float64
form_field16    float64
form_field17    float64
form_field18    float64
form_field19    float64
form_field20    float64
form_field21    float64
form_field22    float64
form_field23    float64
form_field24    float64
form_field25    float64
form_field26    float64
form_field27    float64
form_field28    float64
form_field29    float64
form_field30    float64
form_field31    float64
form_field32    float64
form_field33    float64
form_field34    float64
form_field35    float64
form_field36    float64
form_field37    float64
form_field38    float64
form_field39    float64
form_field40    float64
form_field41    

In [16]:
cat= ('form_field47', 'Applicant_ID', 'default_status')
for x in cat:
  le= LabelEncoder()
  train[x]= le.fit_transform(train[x].values)
  

In [17]:
cats= ('form_field47', 'Applicant_ID')
for y in cats:
  le= LabelEncoder()
  test[y]= le.fit_transform(test[y].values)
  

## **FEATURE** **ENGINEERING**

In [18]:
train.dtypes

Applicant_ID        int64
form_field1       float64
form_field2       float64
form_field3       float64
form_field4       float64
form_field5       float64
form_field6       float64
form_field7       float64
form_field8       float64
form_field9       float64
form_field10      float64
form_field11      float64
form_field12      float64
form_field13      float64
form_field14        int64
form_field15      float64
form_field16      float64
form_field17      float64
form_field18      float64
form_field19      float64
form_field20      float64
form_field21      float64
form_field22      float64
form_field23      float64
form_field24      float64
form_field25      float64
form_field26      float64
form_field27      float64
form_field28      float64
form_field29      float64
form_field30      float64
form_field31      float64
form_field32      float64
form_field33      float64
form_field34      float64
form_field35      float64
form_field36      float64
form_field37      float64
form_field38

In [19]:
train['form_field3233']= (train.form_field32 + train.form_field33) / 2
train.form_field3233

0         530.0000
1        1285.7500
2          35.3333
3         264.9167
4         136.5000
           ...    
55995     442.8333
55996     128.3333
55997     127.1667
55998      80.7500
55999     705.4167
Name: form_field3233, Length: 56000, dtype: float64

In [20]:
test['form_field3233']= (test.form_field32 + test.form_field33) / 2
test.form_field3233

0         119.9167
1         617.7500
2       -4995.5833
3         128.5000
4        1202.3333
           ...    
23995      12.1667
23996     300.0833
23997     103.6667
23998      32.3333
23999     829.4167
Name: form_field3233, Length: 24000, dtype: float64

In [21]:
train= train.drop(['form_field32', 'form_field33', 'Applicant_ID'], axis=1)
test= test.drop(['form_field32', 'form_field33', 'Applicant_ID'], axis= 1)

## **MODELLING**

In [23]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [24]:
X= train[features]
y= train['default_status']


In [25]:
#train_x, val_x, train_y, val_y= train_test_split(X, y, test_size= 0.3,stratify= y, random_state= 42)

## RGF 1 MODEL

In [26]:
rgfparams={
    'max_leaf':1000,
    'algorithm':"RGF_Sib",
    'test_interval':500,
    'verbose': True,
    'loss': 'Log',
    'l2': 0.1,
    'reg_depth':1,
    'sl2':0.01,
}    

In [27]:
skf=12
strata = StratifiedKFold(skf)

In [28]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [29]:
scoring= []
output= 0
test_result = []

for i, (tr_idx, vr_idx) in enumerate(strata.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    rgf =RGFClassifier(**rgfparams)
    #multi= multioutput.MultiOutputClassifier(rgf, n_jobs= 6)
    rgf.fit(xtrain, ytrain)
    
    p1 = rgf.predict_proba(xval)[:, 1]
    sc1 = metric(yval, p1)
    scoring.append(sc1)
    output += sc1/skf
    
    pred1 = rgf.predict_proba(test)[:, 1]
    test_result.append(pred1)

    print('Fold {} : {}'.format(i, sc1))


print('RGF Avg log : ', output)


n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF_Sib
   train_x_fn=/tmp/rgf/4cc1c1b9-63d1-4a7d-9f89-360822feccda1.train.data.x
   train_y_fn=/tmp/rgf/4cc1c1b9-63d1-4a7d-9f89-360822feccda1.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/4cc1c1b9-63d1-4a7d-9f89-360822feccda1.model
--------------------
Wed Sep 30 09:49:07 2020: Reading training data ... 
Wed Sep 30 09:49:08 2020: Start ... #train=51333
--------------------
Forest-level: 
   loss=Log
   max_leaf_forest=1000
   max_tree=500
   opt_interval=100
   test_interval=500
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
Turning on Force_to_refresh_all
-------------
Training data: 49x51333, nonzero_ratio=0.7621; managed as dense data.
-------------
Optimization: 
   loss=Log
   num_iteration_opt=5
   reg_L2=0.1
   opt_stepsize=0.5
   max_delta=1
Tree-level: min_pop=10
Node split: reg_L2=0.01
--------------------
Wed Sep 30 09:49:15 2020: Calling optimizer with 31 trees and 1

In [30]:
f"{skf} fold CV, score: {output}"

'12 fold CV, score: 0.8396798602791472'

In [31]:
rgf_prediction = pd.DataFrame(test_result).T

In [32]:
rgf_prediction.columns = ['fold_'+ str(i) for i in range(1, skf + 1)]

In [33]:
rgf_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12
0,0.307382,0.273013,0.305592,0.307583,0.295395,0.330090,0.298598,0.276636,0.291220,0.353266,0.300042,0.287759
1,0.308051,0.418898,0.372772,0.358380,0.420018,0.400166,0.318572,0.426002,0.383919,0.333774,0.320802,0.353122
2,0.359233,0.384904,0.355796,0.392166,0.417399,0.380006,0.386464,0.391856,0.403290,0.375801,0.354091,0.357549
3,0.762231,0.777941,0.762309,0.738992,0.741698,0.730886,0.743511,0.749191,0.770203,0.707979,0.750646,0.768154
4,0.231994,0.189291,0.178410,0.179513,0.169945,0.185034,0.151812,0.186527,0.178748,0.170818,0.171879,0.181221


In [34]:
what= np.mean(test_result, axis = 0) 

## RGF 2 MODEL

In [112]:
rgfparams1={
    'max_leaf':2000,
    'algorithm':"RGF_Sib",
    'test_interval':200,
    'verbose': True,
    'loss': 'Log',
    #'reg_L2': 0.1,
}    

In [113]:
skf1=12
strata1 = StratifiedKFold(skf1)

In [114]:
scoring1= []
output1 = 0
test_result1 = []

for i, (tr_idx, vr_idx) in enumerate(strata1.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    rgf1 =RGFClassifier(**rgfparams1)
    rgf1.fit(xtrain, ytrain)
    
    p2 = rgf1.predict_proba(xval)[:, 1]
    sc2 = metric(yval, p2)
    scoring.append(sc2)
    output1 += sc2/skf1
    
    pred2 = rgf1.predict_proba(test)[:, 1]
    test_result1.append(pred2)

    print('Fold {} : {}'.format(i, sc2))


print('RGF Avg log : ', output1)


n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF_Sib
   train_x_fn=/tmp/rgf/173cda19-72ff-420f-b44f-c5a81157eff325.train.data.x
   train_y_fn=/tmp/rgf/173cda19-72ff-420f-b44f-c5a81157eff325.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/173cda19-72ff-420f-b44f-c5a81157eff325.model
--------------------
Wed Sep 30 13:36:48 2020: Reading training data ... 
Wed Sep 30 13:36:49 2020: Start ... #train=51333
--------------------
Forest-level: 
   loss=Log
   max_leaf_forest=2000
   max_tree=1000
   opt_interval=100
   test_interval=200
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
Turning on Force_to_refresh_all
-------------
Training data: 49x51333, nonzero_ratio=0.7621; managed as dense data.
-------------
Optimization: 
   loss=Log
   num_iteration_opt=5
   reg_L2=0.1
   opt_stepsize=0.5
   max_delta=1
Tree-level: min_pop=10
Node split: reg_L2=0.1
--------------------
Wed Sep 30 13:36:55 2020: Calling optimizer with 34 trees an

In [115]:
f"{skf1} fold CV, score: {output1}"

'12 fold CV, score: 0.8390778008708184'

In [116]:
rgf_prediction1 = pd.DataFrame(test_result1).T

In [117]:
rgf_prediction1.columns = ['fold_'+ str(i) for i in range(1, skf1 + 1)]

In [118]:
rgf_prediction1.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12
0,0.259425,0.332512,0.231874,0.240532,0.232260,0.282643,0.285522,0.233686,0.243239,0.291406,0.257139,0.248091
1,0.328595,0.329211,0.314929,0.304411,0.377457,0.320786,0.321050,0.332041,0.331945,0.267068,0.284686,0.337446
2,0.337226,0.404193,0.384880,0.439359,0.409128,0.366160,0.394148,0.393578,0.356405,0.378690,0.366933,0.376194
3,0.826844,0.876511,0.877172,0.862815,0.847409,0.837873,0.830777,0.840824,0.795431,0.834711,0.872909,0.810813
4,0.138791,0.132081,0.158277,0.187520,0.140637,0.146771,0.151001,0.203995,0.180332,0.166998,0.156612,0.136128


In [119]:
we= np.mean(test_result1, axis = 0)

## LIGHTGBM 1 MODEL

In [43]:
lgparams={
    'learning_rate': 0.01,
    'min_child_samples':30,
    'n_estimators': 1000,
    'subsample': 0.6,
}

In [44]:
skflg = 10
lgfold= StratifiedKFold(skflg)

In [45]:
scored= []
result = 0
lgbmtest= []

for i, (tr_idx, vr_idx) in enumerate(lgfold.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    lgb =LGBMClassifier(**lgparams)
    lgb.fit(xtrain, ytrain)
    
    p3 = lgb.predict_proba(xval)[:, 1]
    sc3 = metric(yval, p3)
    scored.append(sc3)
    result += sc3/skflg
    
    pred3 = lgb.predict_proba(test)[:, 1]
    lgbmtest.append(pred3)

    print('Fold {} : {}'.format(i, sc3))


print('LGB Avg log : ', result)


Fold 0 : 0.8296961051294085
Fold 1 : 0.8447598542866549
Fold 2 : 0.838849412353554
Fold 3 : 0.8331040285038236
Fold 4 : 0.8413890129267904
Fold 5 : 0.8417856211953627
Fold 6 : 0.8203528607009773
Fold 7 : 0.8406231985293102
Fold 8 : 0.8542306806490674
Fold 9 : 0.8457087761445976
LGB Avg log :  0.8390499550419547


In [46]:
f"{skflg} fold CV, score: {result}"

'10 fold CV, score: 0.8390499550419547'

In [47]:
lg_prediction = pd.DataFrame(lgbmtest).T

In [48]:
lg_prediction.columns = ['fold_'+ str(i) for i in range(1, skflg + 1)]

In [49]:
lg_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10
0,0.303768,0.300854,0.332135,0.294336,0.288134,0.325244,0.326991,0.286847,0.252380,0.306025
1,0.394043,0.424809,0.444375,0.331190,0.439489,0.329674,0.385019,0.318566,0.311502,0.287831
2,0.334267,0.347374,0.391714,0.412554,0.399718,0.376452,0.401434,0.374593,0.361591,0.383529
3,0.722740,0.675266,0.732303,0.735703,0.708004,0.720927,0.712407,0.736743,0.679731,0.697285
4,0.150314,0.176193,0.147949,0.154156,0.170047,0.129853,0.153334,0.113179,0.211109,0.131162


In [50]:
need= np.mean(lgbmtest, axis = 0) 

## LIGHTGBM 2 MODEL

In [51]:
lgparams1={
    'learning_rate': 0.01,
    'min_child_samples':30,
    'n_estimators': 1200,
    'subsample': 0.8,
}

In [52]:
skflg1 = 10
lgfold1= StratifiedKFold(skflg)

In [53]:
scored1= []
result1 = 0
lgbmtest1= []

for i, (tr_idx, vr_idx) in enumerate(lgfold1.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    lgb1 =LGBMClassifier(**lgparams1)
    lgb1.fit(xtrain, ytrain)
    
    p4 = lgb1.predict_proba(xval)[:, 1]
    sc4 = metric(yval, p4)
    scored1.append(sc4)
    result1 += sc4/skflg1
    
    pred4 = lgb1.predict_proba(test)[:, 1]
    lgbmtest1.append(pred4)

    print('Fold {} : {}'.format(i, sc4))


print('LGB Avg log : ', result1)


Fold 0 : 0.8303049400659783
Fold 1 : 0.8444186997527923
Fold 2 : 0.8389448769817103
Fold 3 : 0.8329324163899745
Fold 4 : 0.8413381329533375
Fold 5 : 0.8416902035851508
Fold 6 : 0.8201578881316008
Fold 7 : 0.8405744984843511
Fold 8 : 0.8542079252298298
Fold 9 : 0.8460013211934321
LGB Avg log :  0.8390570902768157


In [54]:
f"{skflg1} fold CV, score: {result1}"

'10 fold CV, score: 0.8390570902768157'

In [55]:
lg_prediction1 = pd.DataFrame(lgbmtest1).T

In [56]:
lg_prediction1.columns = ['fold_'+ str(i) for i in range(1, skflg1 + 1)]

In [57]:
lg_prediction1.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10
0,0.313168,0.303213,0.336055,0.291686,0.290938,0.329004,0.336107,0.285429,0.258642,0.311589
1,0.343907,0.436492,0.426086,0.313977,0.444367,0.292187,0.346851,0.325399,0.307651,0.264405
2,0.333698,0.345844,0.390403,0.414869,0.406192,0.385097,0.407289,0.374189,0.359804,0.388646
3,0.719382,0.672588,0.738204,0.744923,0.708291,0.732625,0.723768,0.750792,0.688640,0.707214
4,0.148230,0.175929,0.141525,0.151000,0.166289,0.120988,0.153136,0.102645,0.195323,0.130027


In [58]:
gan= np.mean(lgbmtest1, axis = 0)

### CATBOOST 1 MODEL

In [59]:

n_skf = 20
kf = StratifiedKFold(n_skf)

seed = 3500

params = {
    'n_estimators': 4000,
    'learning_rate': 0.01,
    'objective': 'CrossEntropy',
    'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 200,
    'max_depth':10
}

In [60]:
score_list = []
score = 0
test_oofs = []

for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    model = CatBoostClassifier(**params)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p5 = model.predict_proba(xval)[:, 1]
    sc5 = metric(yval, p5)
    score_list.append(sc5)
    score += sc5/n_skf
    
    pred5 = model.predict_proba(test)[:, 1]
    test_oofs.append(pred5)

    print('Fold {} : {}'.format(i, sc5))


print('Avg log : ', score)

0:	test: 0.7893431	best: 0.7893431 (0)	total: 246ms	remaining: 16m 24s
100:	test: 0.8198989	best: 0.8198989 (100)	total: 17.3s	remaining: 11m 8s
200:	test: 0.8232414	best: 0.8232414 (200)	total: 33.8s	remaining: 10m 38s
300:	test: 0.8245504	best: 0.8245504 (300)	total: 50.2s	remaining: 10m 17s
400:	test: 0.8256444	best: 0.8256983 (396)	total: 1m 6s	remaining: 9m 58s
500:	test: 0.8266328	best: 0.8266901 (485)	total: 1m 22s	remaining: 9m 39s
600:	test: 0.8272306	best: 0.8272306 (600)	total: 1m 39s	remaining: 9m 21s
700:	test: 0.8275150	best: 0.8275723 (695)	total: 1m 55s	remaining: 9m 3s
800:	test: 0.8279643	best: 0.8279926 (781)	total: 2m 11s	remaining: 8m 45s
900:	test: 0.8286124	best: 0.8286124 (900)	total: 2m 27s	remaining: 8m 28s
1000:	test: 0.8285372	best: 0.8287533 (925)	total: 2m 47s	remaining: 8m 21s
1100:	test: 0.8286684	best: 0.8287533 (925)	total: 3m 3s	remaining: 8m 3s
1200:	test: 0.8290176	best: 0.8290728 (1167)	total: 3m 19s	remaining: 7m 45s
1300:	test: 0.8291412	best: 0.

In [61]:
f"{n_skf} fold CV, score: {score}"

'20 fold CV, score: 0.8421991928028362'

In [62]:
oof_prediction = pd.DataFrame(test_oofs).T

In [63]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [64]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12,fold_13,fold_14,fold_15,fold_16,fold_17,fold_18,fold_19,fold_20
0,0.318063,0.294620,0.269012,0.312489,0.278199,0.286640,0.300448,0.289249,0.286694,0.262877,0.293209,0.307072,0.303064,0.319703,0.296833,0.292736,0.404800,0.314810,0.288295,0.298700
1,0.442188,0.399139,0.396628,0.370605,0.384493,0.395127,0.438860,0.405740,0.462570,0.416679,0.392696,0.344844,0.399939,0.434003,0.398138,0.387769,0.393601,0.447383,0.430113,0.423454
2,0.372412,0.383661,0.421225,0.377584,0.378087,0.402851,0.390580,0.392182,0.402793,0.383785,0.408298,0.386615,0.410583,0.384909,0.385946,0.384744,0.381298,0.362825,0.384798,0.381192
3,0.750657,0.730253,0.741256,0.769171,0.743679,0.752333,0.761435,0.776364,0.750434,0.727754,0.773213,0.775840,0.771128,0.755780,0.741140,0.754565,0.657396,0.731555,0.753315,0.744417
4,0.137783,0.111575,0.154336,0.143392,0.148985,0.133358,0.130782,0.157331,0.154180,0.147780,0.142480,0.146140,0.135301,0.154303,0.118960,0.139692,0.217613,0.131911,0.147424,0.140901


In [65]:
gan1= np.mean(test_oofs, axis= 0)

## CATBOOST 2 MODEL

In [66]:
n_skf1 = 22
kf1 = StratifiedKFold(n_skf1)

seed1 = 2020

params1 = {
    'n_estimators': 4000,
    'learning_rate': 0.01,
    'objective': 'CrossEntropy',
    'eval_metric':'AUC',
    'random_seed': seed1,
    'early_stopping_rounds': 200,
    'max_depth':10
}

In [67]:
score_list1 = []
score1 = 0
test_oofs1 = []

for i, (tr_idx, vr_idx) in enumerate(kf1.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    model1 = CatBoostClassifier(**params1)
    model1.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p6 = model1.predict_proba(xval)[:, 1]
    sc6 = metric(yval, p6)
    score_list.append(sc6)
    score1 += sc6/n_skf1
    
    pred6 = model1.predict_proba(test)[:, 1]
    test_oofs1.append(pred6)

    print('Fold {} : {}'.format(i, sc6))


print('Avg log : ', score1)

0:	test: 0.7926135	best: 0.7926135 (0)	total: 165ms	remaining: 10m 59s
100:	test: 0.8150208	best: 0.8150208 (100)	total: 16.6s	remaining: 10m 39s
200:	test: 0.8189426	best: 0.8189785 (199)	total: 32.8s	remaining: 10m 19s
300:	test: 0.8208775	best: 0.8208866 (298)	total: 48.9s	remaining: 10m 1s
400:	test: 0.8218324	best: 0.8218424 (398)	total: 1m 5s	remaining: 9m 46s
500:	test: 0.8225377	best: 0.8225544 (499)	total: 1m 21s	remaining: 9m 28s
600:	test: 0.8229943	best: 0.8230377 (598)	total: 1m 37s	remaining: 9m 11s
700:	test: 0.8236771	best: 0.8236871 (699)	total: 1m 53s	remaining: 8m 55s
800:	test: 0.8240802	best: 0.8241962 (784)	total: 2m 9s	remaining: 8m 37s
900:	test: 0.8243073	best: 0.8243799 (885)	total: 2m 25s	remaining: 8m 20s
1000:	test: 0.8250994	best: 0.8251286 (993)	total: 2m 41s	remaining: 8m 4s
1100:	test: 0.8249884	best: 0.8251286 (993)	total: 2m 57s	remaining: 7m 48s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8251286071
bestIteration = 993

Shrin

In [68]:
f"{n_skf1} fold CV, score: {score1}"

'22 fold CV, score: 0.8419967364225392'

In [69]:
oof_prediction1 = pd.DataFrame(test_oofs1).T

In [70]:
oof_prediction1.columns = ['fold_'+ str(i) for i in range(1, n_skf1+ 1)]

In [71]:
leleyi= np.mean(test_oofs1, axis= 0)

In [72]:
sub= pd.read_csv('/content/drive/My Drive/Bootcamp2020/SampleSubmission.csv')
sub.head()

,Applicant_ID,default_status
0,Apcnt_1000032,1
1,Apcnt_1000048,1
2,Apcnt_1000052,1
3,Apcnt_1000076,1
4,Apcnt_1000080,1


In [136]:
import numpy as np
final_pred= np.array([])
for i in range (0, len(test)):
  sum= (what[i]+ we[i] + need[i]+ gan[i]+ gan1[i]+leleyi[i])
  total= sum/6
  final_pred= np.append(final_pred, total)
print(final_pred.shape)
#sub['default_status']= final_pred

KeyError: ignored

In [121]:
sub['default_status']= final_pred

In [122]:
sub.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.296600
1,Apcnt_1000048,0.371708
2,Apcnt_1000052,0.382908
3,Apcnt_1000076,0.756277
4,Apcnt_1000080,0.156050


In [125]:
sub.to_csv('lastlast.csv', index = False)

In [126]:
#X= train.drop(['default_status'], axis=1)
#y= train['default_status']

In [142]:
what= DataFrame(what)
what.to_csv('kilofe.csv', index= False)
what.reset_index(drop= True, inplace=True)
what.head()

,0
0,0.302214
1,0.367873
2,0.379880
3,0.750312
4,0.181266


In [130]:
we= DataFrame(we)
we.to_csv('awa.csv', index= False)
we.head()

,0
0,0.261527
1,0.320802
2,0.383908
3,0.842841
4,0.158262


In [131]:
need= DataFrame(need)
need.to_csv('mofe.csv', index= False)
need.head()

,0
0,0.301671
1,0.366650
2,0.378323
3,0.712111
4,0.153730


In [132]:
gan= DataFrame(gan)
gan.to_csv('yes', index= False)
gan.head()

,0
0,0.305583
1,0.350132
2,0.380603
3,0.718643
4,0.148509


In [133]:
gan1= DataFrame(gan1)
gan1.to_csv('yes1', index= False)
gan1.head()

,0
0,0.300876
1,0.408199
2,0.388818
3,0.748084
4,0.144711


In [134]:
leleyi= DataFrame(leleyi)
leleyi.to_csv('thisone.csv', index= False)
leleyi.head()

,0
0,0.307730
1,0.416590
2,0.385919
3,0.765669
4,0.149820


In [152]:
add= gan1+ leleyi
add= add/2
add.head


<bound method NDFrame.head of               0
0      0.304303
1      0.412394
2      0.387369
3      0.756877
4      0.147266
...         ...
23995  0.693644
23996  0.247931
23997  0.280831
23998  0.549258
23999  0.183678

[24000 rows x 1 columns]>

In [153]:
#sub['default_status']= add

In [155]:
#sub.to_csv('oda.csv', index= False)

In [79]:
#train_x, val_x, train_y, val_y= train_test_split(X, y, test_size= 0.3,stratify= y, random_state= 42)

In [80]:
#from catboost import CatBoostClassifier
#cb= CatBoostClassifier()
#model1= LGBMClassifier(learning_rate = 0.01,
                            #min_child_samples = 30,
                            #n_estimators = 1000,
                            #subsample = 0.6)
#gb= GradientBoostingClassifier(n_estimators= 600, learning_rate=0.1, random_state=0)


In [81]:
#from sklearn.ensemble import VotingClassifier

In [82]:
#model = VotingClassifier(estimators=[('lgbm', model1),('gb', model2),('cb', model3)], voting= 'soft')

In [83]:
#model.fit(train_x, train_y)

In [84]:
#pred= model.predict_proba(val_x)[:,1]
#preds= model.predict(val_x)

In [85]:
#print('model:', roc_auc_score(val_y, pred))


In [86]:
#score = accuracy_score(val_y,preds)
#bin_clf_rep = classification_report(val_y,preds, zero_division=1)
#print("Accuracy :", score)
#print(bin_clf_rep)

In [87]:
#model.fit(X,y)

In [88]:
#testi= model.predict_proba(test)[:, 1]

In [89]:
#sub= pd.read_csv('/content/drive/My Drive/Bootcamp2020/SampleSubmission.csv')
#sub.head()

In [90]:
#sub['default_status']= testi

In [91]:
#sub.to_csv('me.csv', index= False)

In [92]:
#rf.fit(train_x, train_y)
#pred= rf.predict_proba(val_x)
#y_pred = rf.predict(val_x)
#print('rf:', roc_auc_score(val_y, pred[:,1]))
#cv_sc = cross_val_score(rf,X, y ,cv=5).mean()
#bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#print(bin_clf_rep)
#print(cv_sc)

In [93]:
#gb= GradientBoostingClassifier(n_estimators=500, learning_rate= 0.01, random_state=0)
#gb.fit(train_x, train_y)
#pred= gb.predict_proba(val_x)
#y_pred = gb.predict(val_x)
#print('gb:', roc_auc_score(val_y, pred[:,1]))
#bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#cv_scor = cross_val_score(gb,X, y ,cv=5).mean()
#print(cv_scor)
#print(bin_clf_rep)

gb: 0.8324774223913745


In [94]:
#xb= XGBClassifier()
#xb.fit(train_x, train_y)
#pred= xb.predict_proba(val_x)
#y_pred = xb.predict(val_x)
#print('xb:', roc_auc_score(val_y, pred[:,1]))
#bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#cv_scor = cross_val_score(xb,X, y ,cv=5).mean()
#print(cv_scor)
#print(bin_clf_rep)

In [95]:
#cb=CatBoostClassifier()
#cb.fit(train_x, train_y)
#pred= cb.predict_proba(val_x)
#y_pred = cb.predict(val_x)
#print('cb:', roc_auc_score(val_y, pred[:,1]))
#bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#c#v_scor = cross_val_score(cb,X, y ,cv=5).mean()
#print(cv_scor)
#print(bin_clf_rep)

In [96]:
#lgb= LGBMClassifier()
#lgb.fit(train_x,train_y)
#preds = lgb.predict(val_x)
#prob= lgb.predict_proba(val_x)

#roc_score = roc_auc_score(val_y, prob[:,1])
#cv_score = cross_val_score(lgb,X,y,cv=5).mean()
#score = accuracy_score(val_y,preds)
#bin_clf_rep = classification_report(val_y,preds, zero_division=1)
#print("Accuracy :", score)
#print("CV Score :", cv_score)
#print("AUC Score : ", roc_score)
#print(bin_clf_rep)
#print(confusion_matrix(y_test,y_pred))

#lgb

In [97]:
#lgbm_params = {"n_estimators" : [100, 200, 500, 1000, 2000],
               #"subsample" : [0.6, 0.8, 1.0],
               #"learning_rate" : [0.1, 0.01, 0.02, 0.03],
               #"min_child_samples" : [5, 10, 20, 30]}

In [98]:
#from sklearn.model_selection import GridSearchCV
#lgbm_cv_model = GridSearchCV(lgbm_m, 
                             #lgbm_params, 
                             #cv = 5,
                             #verbose = 1,
                             #n_jobs = -1)

In [99]:
##lgbm_cv_model.fit(train_x, train_y)

In [100]:
#lgbm_cv_model.best_params_

In [101]:
#lgbm_m = LGBMClassifier(learning_rate = 0.01,
                            #min_child_samples = 30,
                            #n_estimators = 1000,
                            #subsample = 0.6)
#lgbm_m.fit(train_x,train_y)
#y_pred = lgbm_m.predict(val_x)
#proba = lgbm_m.predict_proba(val_x)
#roc_score = roc_auc_score(val_y, proba[:,1])
#v_score = cross_val_score(lgbm_m,X, y ,cv=5).mean()
#score = accuracy_score(val_y,y_pred)
#bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#print(name[0:name.find("(")])
#print("Accuracy :", score)
#print("CV Score :", cv_score)
#print("AUC Score : ", roc_score)
#print(bin_clf_rep)
#print(confusion_matrix(y_test,y_pred))

Accuracy : 0.8092857142857143
CV Score : 0.8079285714285713
AUC Score :  0.8396048614793689


In [102]:
#lgbm_m.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=30, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

In [103]:
#cb.fit(X,y)

In [104]:
#gb.fit(X,y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [105]:
#testi= lgbm_m.predict_proba(test)[:, 1]

In [106]:
#testii= gb.predict_proba(test)[:, 1]

In [107]:
#test_yhat= cb.predict_proba(test)[:, 1]

In [108]:
#code= pd.read_csv('/content/drive/My Drive/Bootcamp2020/code.csv')
#code

,Applicant_ID,default_status
0,Apcnt_1000032,0.315847
1,Apcnt_1000048,0.392468
2,Apcnt_1000052,0.382775
3,Apcnt_1000076,0.736655
4,Apcnt_1000080,0.186125
...,...,...
23995,Apcnt_999940,0.688645
23996,Apcnt_999956,0.291725
23997,Apcnt_999976,0.276531
23998,Apcnt_999984,0.524821


In [109]:
#sub['default_status'] = testii


In [110]:
#sub['default_status'] = testi

In [111]:
#ensemblepred= (testi* 0.3+ testii *0.2 + dec*0.5)
#ensemblepred

NameError: ignored

In [ ]:
#emax= ensemblepred*0.5 + code*0.5

In [ ]:
#sub['default_status'] = ensemblepred

In [ ]:
#sub

In [ ]:
#sub.to_csv('bagging.csv', index= False)